# Công thức nội suy trung tâm Stirling 

## Điều kiện: các mốc là cách đều nhau và số mốc sử dụng là một số lẻ

# 1. Code của chương trình. Khi sử dụng code thì "Run All" và thao tác ở task "Chạy chương trình"

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [11]:


def doc_input(ten_file):
    # Read x and y values from an Excel file (xlsx format)
    df = pd.read_excel(ten_file)

    # Extract x values from the first column
    x = df.iloc[:, 0].dropna().values
    x = np.array(x, dtype=float)

    # Check if y column exists or is empty
    if df.shape[1] < 2 or df.iloc[:, 1].isnull().all():
        # If y is empty, calculate y using function f(x)
        y = f(x)
        y = np.array(y, dtype=float)
    else:
        # If y column exists, read y values
        y = df.iloc[:, 1].dropna().values
        y = np.array(y, dtype=float)

    return x, y

def hoocnerChia(a, value):
    b = []
    b.append(a[0])
    for i in range(1, len(a)):
        b.append(b[i - 1] * value + a[i])
    return b
def hoocnerNhanBosung(coeffPolyTich, t):
    if len(coeffPolyTich) == 0:
        coeffPolyTich.append(1)
        coeffPolyTich.append(-t)
    else:
        coeffPolyTich.append(0)
        temp = coeffPolyTich.copy()
        for i in range(1, len(coeffPolyTich)):
            coeffPolyTich[i] = coeffPolyTich[i] - t * temp[i - 1]
    return coeffPolyTich
def saiPhan(f_0, f_1, x0, xk, yk):
    if xk > x0:
        temp = f_0.copy()
        f_0.append(0)
        f_0[0] = yk
        for i in range(1, len(f_0)):
            f_0[i] = f_0[i - 1] - temp[i - 1]
        f_1.append(f_0[len(f_0) - 1])
    else:
        temp = f_1.copy()
        f_1.append(0)
        f_1[0] = yk
        for i in range(1, len(f_1)):
            f_1[i] = temp[i - 1] - f_1[i - 1]
        f_0.append(f_1[len(f_1) - 1])
    return f_0, f_1
def stirlingFormula(heSoDaThuc, coeffTich, f_0, f_1, k, x0, x1, x2, y1, y2, giaiThua_k):
    #da thuc theo an t; k được hiểu là mốc ở vị trí mốc thứ k thêm vào (vị trí y2)
    #k luôn là số chẵn vì số mốc sử dụng là số lẻ
    #bat dau chay ham voi k le bat dau tu 1
    f_0, f_1 = saiPhan(f_0, f_1, x0, x2, y2)
    f_0, f_1 = saiPhan(f_0, f_1, x0, x1, y1)

    #su dung dat u = t^2 => t^2; t^2(t^2 - 1); t^2(t^2 - 1)(t^2 - 4); t^2(t^2 - 4)(t^2 - 9)
    coeffTich = hoocnerNhanBosung(coeffTich, (k/2 - 1/2)**2)

    giaiThua_k *= len(heSoDaThuc)
    c_le = (f_1[len(f_1) - 2] + f_0[(len(f_0) - 2)]) / (2 * giaiThua_k)
    giaiThua_k *= len(heSoDaThuc) + 1
    c_chan = f_1[len(f_1) - 1] / giaiThua_k
    
    #tinh he so da thuc voi da thuc tich them vao la da thuc bac le
    heSoDaThucCu = heSoDaThuc.copy()
    heSoDaThuc.append(0)
    for i in range(1, len(heSoDaThuc)):
        if i % 2 == 0:
            heSoDaThuc[i] = heSoDaThucCu[i - 1] + c_le * coeffTich[int((i + 1) / 2)]
        else:
            heSoDaThuc[i] = heSoDaThucCu[i - 1]
    heSoDaThuc[0] = c_le

    #tinh he so da thuc voi da thuc tich them vao la da thuc bac chan
    heSoDaThucCu = heSoDaThuc.copy()
    heSoDaThuc.append(0)
    for i in range(1, len(heSoDaThuc)):
        if i % 2 == 0:
            heSoDaThuc[i] = heSoDaThucCu[i - 1] + c_chan * coeffTich[int((i + 1) / 2)]
        else:
            heSoDaThuc[i] = heSoDaThucCu[i - 1]
    heSoDaThuc[0] = c_chan

    return heSoDaThuc, coeffTich, f_0, f_1, giaiThua_k

def sortData(path):
    # Read data based on file extension
    if path.endswith('.csv'):
        data = pd.read_csv(path)
        dataX = data['x'].values
        dataY = data['y'].values
    elif path.endswith('.txt'):
        dataX, dataY = doc_input("input.txt")
    elif path.endswith('.xlsx'):
        data = pd.read_excel(path)
        dataX = data['x'].values
        dataY = data['y'].values

    # Selection sort for sorting dataX and adjusting dataY accordingly
    for i in range(len(dataX)):
        indexMin = i
        for j in range(i + 1, len(dataX)):
            if dataX[j] < dataX[indexMin]:
                indexMin = j
        if indexMin != i:
            dataX[indexMin], dataX[i] = dataX[i], dataX[indexMin]
            dataY[indexMin], dataY[i] = dataY[i], dataY[indexMin]

    return dataX, dataY

def checkSoMoc(dataX, soMoc, x0):
    h = dataX[1] - dataX[0]
    m = round((x0 - dataX[0])/h)
    if m <= len(dataX)/2:
        n = 2 * m + 1
    else:
        n = 2 * (len(dataX) - m) + 1
    if n > len(dataX):
        n = len(dataX)
        
    #check xem so moc chon co hop le hay khong. So moc noi suy su dung phai la so le
    if soMoc > n or soMoc <= 0 or soMoc % 2 == 0:
        print("so luong moc lua chon la khong hop le")
        print("Nhap lai so moc. So moc phai la so \"le\" thuoc trong khoang tu:", 0, "->", n)
    else:
        print("so luong moc la hop le. Tiep tuc chuong trinh")
def sortGauss(dataX, dataY, x0, soMoc):
    x = []
    y = []
    m = round((x0 - dataX[0]) / (dataX[1] - dataX[0]))
    for i in range(soMoc):
        if i % 2 == 1:
            k = int((i + 1)/2)
            x.append(dataX[m + k])
            y.append(dataY[m + k])
        else:
            k = int(-(i + 1)/2)
            x.append(dataX[m + k])
            y.append(dataY[m + k])
    return x, y

In [12]:
def main(dataX, dataY, x0, soMoc):
    heSoDaThuc = []
    coeffTich = []
    f_0 = [] 
    f_1 = [] 
    giaiThua_k = 1
    h = dataX[1] - dataX[0]
    with open("output.txt", "w") as out:
        #buoc co so
        x, y = sortGauss(dataX, dataY, x0, soMoc)
        print("Các mốc đã sử dụng là: \nx = {0}\ny = {1}".format(x, y))
        heSoDaThuc.append(y[0])
        f_0.append(y[0])
        f_1.append(y[0])
        out.writelines("\n\nMoc noi suy ban dau duoc chon la: ({}, {})".format(x[0], y[0]))
        #cac buoc them dan 2 moc tiep theo
        for i in range(1, soMoc, 2):
            heSoDaThuc, coeffTich, f_0, f_1, giaiThua_k = stirlingFormula(heSoDaThuc, coeffTich, f_0, f_1, i, x0, x[i], x[i + 1], y[i], y[i + 1], giaiThua_k)
            
            out.writelines("\n\n--------------------------------------------------------------------------------------")
            out.writelines("\nDa thuc moi sau khi them moc 2 noi suy: ({}, {}); ({}, {}) ".format(x[i], y[i], x[i + 1], y[i + 1]))
            out.write("\nTap cac moc noi suy: \t")
            for j in range(i + 2):
                out.write("({}, {}); \t".format(x[j], y[j]))
            out.write("\nDa thuc bac {}: \nP(t) =: ".format(i + 1))
            for j in range(len(heSoDaThuc)):
                out.write("{} * x^{} \t + \t".format(heSoDaThuc[j], len(heSoDaThuc) - 1 - j))
            out.write("\n\nThu lai\n")
            for j in range(i + 2):
                out.writelines("\nTai du lieu x = {}".format(x[j]))
                a = hoocnerChia(heSoDaThuc, (x[j] - x[0])/h)
                out.writelines("\nPn(x) - y = {}".format(a[len(heSoDaThuc) - 1] - y[j]))
    return heSoDaThuc, coeffTich, f_0, f_1, giaiThua_k

# 2. Chạy chương trình

In [13]:
# Dữ liệu đầu vào
x, y = doc_input("input.xlsx")

# Mốc ban đầu:
x0 = 2.6

# Số lượng mốc sử dụng:
soMoc = 5

heSodaThuc, coeffTich, f_0, f_1, giaiThua_k =  main(x, y, x0, soMoc)
print("\nĐa thức nội suy có bậc là: ", len(heSodaThuc) - 1)
print("Hệ số của đa thức nội suy là: ")
for i in range(len(heSodaThuc)):
    print("{0} x^{1}".format(heSodaThuc[i], len(heSodaThuc)-1-i))

Các mốc đã sử dụng là: 
x = [np.float64(3.0), np.float64(4.0), np.float64(2.0), np.float64(5.0), np.float64(6.0)]
y = [np.float64(1.0), np.float64(2.0), np.float64(0.8), np.float64(3.0), np.float64(4.0)]

Đa thức nội suy có bậc là:  4
Hệ số của đa thức nội suy là: 
-0.17500000000000002 x^4
-0.31666666666666665 x^3
0.5750000000000001 x^2
0.9166666666666666 x^1
1.0 x^0


In [14]:
# Điểm cần tính giá trị => cần chọn mốc ban đầu x0 gần xk
xk = 2.4

h = x[1] - x[0]
t = (xk - x0)/h
print("Giá trị của đa thức nội suy tại xk = {0}\nLà: {1}".format(xk, hoocnerChia(heSodaThuc, t).pop()))

Giá trị của đa thức nội suy tại xk = 2.4
Là: 0.8419199999999999
